# Predictive Analysis

In [1]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import when, col, explode, max, avg, count, udf, expr, size, datediff
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier, MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, ArrayType, StringType

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
spark = SparkSession.builder.appName("Forum Question Analyzer") \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.17.0")\
    .getOrCreate()

posts = spark.read \
    .format("com.databricks.spark.xml") \
    .option("rootTag", "posts") \
    .option("rowTag", "row") \
    .load("tex.stackexchange.com/Posts.xml") \
    .alias('posts')
users = spark.read \
    .format("com.databricks.spark.xml") \
    .option("rootTag", "users") \
    .option("rowTag", "row") \
    .load("tex.stackexchange.com/Users.xml") \
    .alias('users')
tags = spark.read \
    .format("com.databricks.spark.xml") \
    .option("rootTag", "tags") \
    .option("rowTag", "row") \
    .load("tex.stackexchange.com/Tags.xml") \
    .alias('tags')

### Feature Extraction

In [6]:
posts = posts.filter(posts._PostTypeId==1)
questions = posts.withColumn("_Tags", expr("split(substring(_Tags, 2, length(_Tags) - 2), '><')"))\
            .withColumn("_Body", size(expr("split(_Body, ' ')")))\
            .withColumn("_Title", size(expr("split(_Title, ' ')")))

questions = questions.join(users, questions._OwnerUserId == users._Id).select(
    questions._Id.alias("QuestionId"),
    questions._Body.alias("BodyLength"),
    questions._Title.alias("TitleLength"),
    questions._Tags.alias("TagsCounts"),
    size(questions._Tags).alias("NumberOfTags"),
    users._Id.alias("OwnerId"),
    users._DownVotes.alias("OwnerDownVotes"),
    users._UpVotes.alias("OwnerUpVotes"),
    users._Reputation.alias("OwnerReputation"),
    users._Views.alias("OwnerViews"),
    (questions._CreationDate - users._CreationDate).cast("integer").alias("UserExperience"),
    when(col("_AcceptedAnswerId").isNull(), 0).otherwise(1).alias("Accepted")
)

In [13]:
#Fast way to change tag names to tag counts from Tags table

tag_counts = tags.select("_TagName", "_Count").rdd.collectAsMap()

def replace_tags_with_counts(tags):
    return [tag_counts.get(tag, 0) for tag in tags]
print(tag_counts)
print(replace_tags_with_counts(["enumerate", "geometry"]))
replace_tags_with_counts_udf = udf(replace_tags_with_counts, IntegerType())

questions = questions.withColumn("TagsCounts", max(replace_tags_with_counts_udf(questions.TagsCounts)))

{'lyx': 3033, 'include': 623, 'compiling': 2035, 'labels': 1679, 'beamer': 11290, 'table-of-contents': 7102, 'subjective': 5, 'learning': 110, 'word-to-latex': 62, 'spelling': 187, 'publishing': 49, 'xetex': 5574, 'luatex': 4131, 'context': 1796, 'brackets': 840, 'tools': 206, 'format-files': 132, 'opentype': 234, 'math-operators': 965, 'programming': 623, 'msword': 247, 'conversion': 534, 'resume': 248, 'fonts': 8795, 'computer-modern': 132, 'graphics': 7013, 'automation': 579, 'document-classes': 1134, 'preamble': 403, 'packages': 3251, 'makefile': 83, 'windows': 832, 'cleanup': 18, 'bibtex': 5750, 'bibliographies': 6940, 'syntax': 361, 'web': 32, 'embedding': 173, 'enumerate': 2364, 'revision-control': 93, 'syntax-checker': 28, 'align': 3998, 'equations': 6023, 'verbatim': 1108, 'pstricks': 2000, 'line-breaking': 2807, 'ubuntu': 596, 'linux': 482, 'notes': 198, 'input': 858, 'macros': 8117, 'miktex': 2539, 'texdoc': 66, 'online': 149, 'links': 512, 'indexing': 1396, 'minipage': 1724

AnalysisException: [MISSING_GROUP_BY] The query does not include a GROUP BY clause. Add GROUP BY or turn it into the window functions using OVER clauses.;
Aggregate [QuestionId#458L, BodyLength#459, TitleLength#460, max(replace_tags_with_counts(TagsCounts#640)#657) AS TagsCounts#659, NumberOfTags#462, OwnerId#463L, OwnerDownVotes#464L, OwnerUpVotes#465L, OwnerReputation#466L, OwnerViews#467L, UserExperience#468, Accepted#469]
+- Project [QuestionId#458L, BodyLength#459, TitleLength#460, replace_tags_with_counts(TagsCounts#550)#639 AS TagsCounts#640, NumberOfTags#462, OwnerId#463L, OwnerDownVotes#464L, OwnerUpVotes#465L, OwnerReputation#466L, OwnerViews#467L, UserExperience#468, Accepted#469]
   +- Project [QuestionId#458L, BodyLength#459, TitleLength#460, replace_tags_with_counts(TagsCounts#461)#549 AS TagsCounts#550, NumberOfTags#462, OwnerId#463L, OwnerDownVotes#464L, OwnerUpVotes#465L, OwnerReputation#466L, OwnerViews#467L, UserExperience#468, Accepted#469]
      +- Project [_Id#9L 

In [9]:
questions.show(5, truncate=False)
# posts_tags = posts.select(col("_Id").alias("_Id"),
#         explode(col('_Tags')).alias("tag"))\
#         .filter(posts._PostTypeId == 1)

# posts_tags_score = \
#         posts_tags.join(tags, posts_tags.tag == tags._TagName)\
#         .select(posts_tags._Id, tags._Count).groupby(posts_tags._Id)\
#                 .agg(max(tags._Count).alias("max_tag_count"),
#                      avg(tags._Count).alias("avg_tag_count"),
#                      count(tags._Count).alias("number_of_tags"))


PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "C:\Users\jurek\AppData\Local\Temp\ipykernel_1084\1988812052.py", line 6, in replace_tags_with_counts
  File "C:\Spark\spark-3.5.0-bin-hadoop3\python\lib\pyspark.zip\pyspark\sql\utils.py", line 174, in wrapped
    return f(*args, **kwargs)
  File "C:\Spark\spark-3.5.0-bin-hadoop3\python\lib\pyspark.zip\pyspark\sql\functions.py", line 687, in max
    return _invoke_function_over_columns("max", col)
  File "C:\Spark\spark-3.5.0-bin-hadoop3\python\lib\pyspark.zip\pyspark\sql\functions.py", line 105, in _invoke_function_over_columns
    return _invoke_function(name, *(_to_java_column(col) for col in cols))
  File "C:\Spark\spark-3.5.0-bin-hadoop3\python\lib\pyspark.zip\pyspark\sql\functions.py", line 105, in <genexpr>
    return _invoke_function(name, *(_to_java_column(col) for col in cols))
  File "C:\Spark\spark-3.5.0-bin-hadoop3\python\lib\pyspa

In [6]:
features = ['title_length', 'question_length', 'author_reputation',
            'max_tag_count', 'avg_tag_count', 'number_of_tags', 'proper_title']
assembler = VectorAssembler(inputCols=features, outputCol="features")

### Data Preparation

### Model Training

In [7]:
train, test = questions.randomSplit([0.7, 0.3], seed=12345)

In [8]:
# Logistic Regression model
lr = LogisticRegression(labelCol="accepted", featuresCol="features")
lr_pipeline = Pipeline(stages=[assembler, lr])
lr_model = lr_pipeline.fit(train)

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "c:\Users\jurek\anaconda3\lib\socket.py", line 707, in readinto
    raise
socket.timeout: timed out


In [ ]:
# Random Forest model
rf = RandomForestClassifier(labelCol="accepted", featuresCol="features", numTrees=10)
rf_pipeline = Pipeline(stages=[assembler, rf])
rf_model = rf_pipeline.fit(train)

In [ ]:
# Gradient Boosting model
gbt = GBTClassifier(labelCol="accepted", featuresCol="features", maxIter=10)
gbt_pipeline = Pipeline(stages=[assembler, gbt])
gbt_model = gbt_pipeline.fit(train)

In [ ]:
# Neural Network model
layers = [len(features), 10, 5, 2]  # Adjust layer sizes as needed
nn = MultilayerPerceptronClassifier(labelCol="accepted", featuresCol="features", layers=layers, blockSize=128, seed=1234)
nn_pipeline = Pipeline(stages=[assembler, nn])
nn_model = nn_pipeline.fit(train)

### Predictions

In [ ]:
lr_predictions = lr_model.transform(test)
rf_predictions = rf_model.transform(test)
gbt_predictions = gbt_model.transform(test)
nn_predictions = nn_model.transform(test)

### Model Evaluation

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="accepted", metricName="accuracy")
print('===== Accuracy =====')
print('Logistic Regression:', evaluator.evaluate(lr_predictions))
print('Random Forest:      ', evaluator.evaluate(rf_predictions))
print('Gradient Boosting:  ', evaluator.evaluate(gbt_predictions))
print('Neural Network:     ', evaluator.evaluate(nn_predictions))

===== Accuracy =====
Logistic Regression: 0.6023578712851592
